In [2]:
import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import plotly.graph_objects as go
import numpy as np
from typing import *
from llama.main import compute_metrics_from_files, DatasetName, compute_metrics_from_list, get_tokenizer
import llama.main as m
from sklearn.metrics import f1_score, auc, roc_curve, confusion_matrix
from PIL import Image
from pandasql import sqldf
from io import BytesIO
import base64
from plotly.graph_objects import Figure
import win32clipboard
import textwrap
import colorsys
import optuna
import tqdm
import re
import random
from sklearn.metrics import accuracy_score, roc_curve, auc
def img_to_clipboard(fig: Figure):
    img_bytes = fig.to_image(format="png")
    image = Image.open(BytesIO(img_bytes))
    
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()
root = Path(f"../../data/baseline_studies/analysis/sel/")

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed
Import of from pytorchtools import EarlyStopping failed


In [ ]:
l = []
files = list(enumerate(root.rglob("*_0.json")))
for i,f in tqdm.tqdm(files):
    algorithm_group: Literal['bayesian_network','svc','knn']
    if "_nb_" in f.stem:
        algorithm_group = 'bayesian_network'
    elif "_svm_knn_" in f.stem:
        if "KNeighborsClassifier" in f.stem:
            algorithm_group = "knn"
        elif "SVC_" in f.stem:
            algorithm_group = "svc"
        else:
            raise Exception
    else:
        raise Exception
    num_samples = int(re.search("([0-9]+)_samples", f.stem).group(1))
            
    with open(f) as fp:
        data = json.load(fp)[0]
    dataset_choice = "eclipse_72k" if "eclipse_72k" in f.stem else "mozilla_200k"
    fixed_params = {k:v for k,v in data.items() if not isinstance(v, list)}
    fixed_params["params"] = fixed_params
    fixed_params["run_id"] = i
    fixed_params["num_samples"] = num_samples
    fixed_params["algorithm_group"] = algorithm_group
    fixed_params["dataset_choice"] = dataset_choice
    infos = {**fixed_params}
    for dataset in ["val","test"]:
        d = [
            {"binary_severity":b, "severity_pred": p, "bug_id": i} 
            for i,(b,p) in enumerate(zip(
                data[f'binary_severity_{dataset}'],
                data[f'y_{dataset}_pred']
                ))]
        threasholds = sorted(np.unique(filter(lambda x:x is not None,data[f'thresholds_{dataset}'])))
        fpr, tpr, thresholds = roc_curve(data[f'binary_severity_{dataset}'], data[f'y_{dataset}_pred'])
        auc_test = auc(fpr, tpr)
        fig = go.Figure()
        y_scores = data[f'y_{dataset}_pred']
        y_true = data[f'binary_severity_{dataset}']
        roc_data = []
        best_accuracy = 0
        best_threshold = -1
        best_data = None

        for i, threshold in enumerate(thresholds):
            y_pred = (y_scores >= threshold).astype(int)
            cm = confusion_matrix(y_true, y_pred)
            accuracy = np.sum(np.diag(cm))/np.sum(cm)*100
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_threshold = threshold
                best_data = y_true, y_pred
        assert best_data is not None
        d = [
            {"binary_severity":y_true, "severity_pred": y_pred, "bug_id": i} for i,(y_true, y_pred) in enumerate(zip(*best_data))]
        cm, f1, _ = compute_metrics_from_list(d, pred_field="severity_pred")
        accuracy = np.sum(np.diag(cm)) / np.sum(cm)
        print(f"{dataset=} {f.stem=} {auc_test=}")
        infos[f"accuracy_{dataset}"] = accuracy
        infos[f"auc_{dataset}"] = accuracy
        infos[f"f1_{dataset}"] = f1
        infos[f"threashold_{dataset}"] = best_threshold
        infos[f"confusion_matrix_{dataset}"] = cm
    l.append(infos)
df = pd.DataFrame(l)

In [3]:
def remove_params(d):
    return {k:v for k,v in d.items() if k != "params"}
df.loc[:,"params"] = df["params"].apply(remove_params)
df.to_json("../../data/baseline_studies/baseline_results_analysed_v2.json")

In [25]:
df = pd.read_json("../../data/baseline_studies/baseline_results_analysed_v2.json")

In [27]:
dataset_choice = "eclipse_72k"
d = df.query(f"dataset_choice == '{dataset_choice}'")
d.sort_values(["num_samples","algorithm_group"],inplace=True)
d = d.rename({"accuracy_val": "Validation accuracy with best threshold", "accuracy_test": "Test accuracy with best threshold"}, axis=1)
fig = px.line(
    d,
    x="num_samples",
    y="auc_val",
    color="algorithm_group",
    title=f"Area under the curve for {dataset_choice}",
    markers=True,
)
for i,trace in enumerate(fig.data):
    trace.name =  trace.name + ' validation'
fig2 = px.line(d, x="num_samples", y="auc_test", color="algorithm_group")
for i,trace in enumerate(fig2.data):
    trace.name = trace.name + ' test'
fig2.update_traces(patch={"line": {"dash": 'dot', 'width': 4}}) 
for i in range(3):
    fig.add_trace(fig2.data[i])
fig.update_layout(
    font=dict(size=16)  # Set the default font size for all text elements in the figure
)
fig.update_yaxes(title_text="Area under the curve")
fig.show()

C:\Users\robin\AppData\Local\Temp\ipykernel_11712\2996298871.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.sort_values(["num_samples","algorithm_group"],inplace=True)


In [4]:
dataset_choice = "eclipse_72k"
d = df.query(f"dataset_choice == '{dataset_choice}' & num_samples == 8000")
d.sort_values(["num_samples","algorithm_group"],inplace=True)
for i,l in d.iterrows():
    conf_matrix_test = l["confusion_matrix_test"]
    algo = l['classifier']
    for conf_matrix_name, conf_matrix_id in zip(["Validation", "Test"],["val","test"]):
        conf_matrix = l[f"confusion_matrix_{conf_matrix_id}"]
        f1 = l[f"f1_{conf_matrix_id}"]
        threashold = l[f"threashold_{conf_matrix_id}"]
        compute_metrics_from_files(
            conf_matrix=conf_matrix,
            f1=f1,
            folder_out=Path("../../data/baseline_studies/"),
            title=f"{dataset_choice} {conf_matrix_name} threashold {threashold:.2f}",
            id=f"{dataset_choice}_{conf_matrix_name}"
        )

C:\Users\robin\AppData\Local\Temp\ipykernel_16784\4158893852.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: U

In [5]:
dataset_choice = "mozilla_200k"
d = df.query(f"dataset_choice == '{dataset_choice}'")
d.sort_values(["num_samples","algorithm_group"],inplace=True)
d = d.rename({"accuracy_val": "Validation accuracy with best threshold", "accuracy_test": "Test accuracy with best threshold"}, axis=1)
fig = px.line(
    d,
    x="num_samples",
    y="auc_val",
    color="algorithm_group",
    title=f"Area under the curve for {dataset_choice}",
    markers=True,
)
for i,trace in enumerate(fig.data):
    trace.name =  trace.name + ' validation'
fig2 = px.line(d, x="num_samples", y="auc_test", color="algorithm_group")
for i,trace in enumerate(fig2.data):
    trace.name = trace.name + ' test'
fig2.update_traces(patch={"line": {"dash": 'dot', 'width': 4}}) 
for i in range(3):
    fig.add_trace(fig2.data[i])
fig.update_layout(
    font=dict(size=16)  # Set the default font size for all text elements in the figure
)
fig.update_yaxes(title_text="Area under the curve")
fig.show()

C:\Users\robin\AppData\Local\Temp\ipykernel_16784\3485171786.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
dataset_choice = "mozilla_200k"
d = df.query(f"dataset_choice == '{dataset_choice}' & num_samples == 8000")
d.sort_values(["num_samples","algorithm_group"],inplace=True)
for i,l in d.iterrows():
    conf_matrix_test = l["confusion_matrix_test"]
    algo = l['classifier']
    for conf_matrix_name, conf_matrix_id in zip(["Validation", "Test"],["val","test"]):
        conf_matrix = l[f"confusion_matrix_{conf_matrix_id}"]
        f1 = l[f"f1_{conf_matrix_id}"]
        threashold = l[f"threashold_{conf_matrix_id}"]
        compute_metrics_from_files(
            conf_matrix=conf_matrix,
            f1=f1,
            folder_out=Path("../../data/baseline_studies/"),
            title=f"{dataset_choice} {conf_matrix_name} threashold {threashold:.2f}",
            id=f"{dataset_choice}_{conf_matrix_name}"
        )

C:\Users\robin\AppData\Local\Temp\ipykernel_16784\3960788679.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:907: U